In [ ]:
# identifying impersonator by identifying author style

In [66]:
import tweepy

# Consumer keys and access tokens, used for OAuth
api_key = 'g5uPIpw80nULQI1gfklv2zrh4'
api_secret = 'cOWvNWxYvPmEZ0ArZVeeVVvJu41QYHdUS2GpqIKtSQ1isd5PJy'
access_token = '49722956-TWl8J0aAS6KTdcbz3ppZ7NfqZEmrwmbsb9cYPNELG'
access_secret = '3eqrVssF3ppv23qyflyAto8wLEiYRA8sXEPSghuOJWTub'


# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth)
my_tweets, other_tweets = [], []

to_get = 500
for status in tweepy.Cursor(api.user_timeline, screen_name='@prof_oz').items():
    text = status._json['text']
    if text[:3] != 'RT ':  # we don't want retweets because they didn't author those!
        my_tweets.append(text)
    else:
        other_tweets.append(text)
    to_get -= 1
    if to_get <=0:
        break

In [67]:
len(real_tweets), len(other_tweets)

(131, 151)

In [68]:
real_tweets[0], other_tweets[0]

(u'@stanleyyork Definitely check out the Grand Bazaar as well as a tour around the Mosques and surrounding caf\xe9s / sho\u2026 https://t.co/ETREtznTgr',
 u'RT @SThornewillvE: This weeks @superdatasci podcast has a lot of really interesting talk about #featureengineering, with @Prof_OZ, the auth\u2026')

In [69]:
import pandas

In [86]:
df = pandas.DataFrame({'text': my_tweets+other_tweets, 'is_mine': [True]*len(my_tweets)+[False]*len(other_tweets)})

df.shape

(386, 2)

In [87]:
df.head(2)

,is_mine,text
0,True,@stanleyyork Definitely check out the Grand Ba...
1,True,12 Exciting Ways You Can Use Voice-Activated T...


In [88]:
df.tail(2)

,is_mine,text
384,False,RT @Variety: BREAKING: #TheInterview will be s...
385,False,RT @ProfLiew: Let's all congratulate Elizabeth...


In [92]:
import numpy as np
np.random.seed(10)

remove_n = int(.1 * df.shape[0])  # remove 10% of rows for validation set

drop_indices = np.random.choice(df.index, remove_n, replace=False)
validation_set = df.iloc[drop_indices]
training_set = df.drop(drop_indices)

In [93]:
validation_set.shape, training_set.shape

((38, 2), (348, 2))

In [94]:
X, y = training_set['text'], training_set['is_mine']

In [167]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [96]:
pipeline_parts = [
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
    
]
simple_pipeline = Pipeline(pipeline_parts)

In [97]:
simple_grid_search_params = {
    "vectorizer__ngram_range": [(1, 1), (1, 3), (1, 5)],
    "vectorizer__analyzer": ["word", "char", "char_wb"],
}

grid_search = GridSearchCV(simple_pipeline, simple_grid_search_params)

grid_search.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), pre...one, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vectorizer__ngram_range': [(1, 1), (1, 3), (1, 5)], 'vectorizer__analyzer': ['word', 'char', 'char_wb']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [98]:
grid_search.best_score_  # best cross validated accuracy

0.896551724137931

In [101]:
model = grid_search.best_estimator_

In [102]:
# % False, % True
model.predict_proba([my_tweets[0]])

array([[2.56519064e-07, 9.99999743e-01]])

In [106]:
from sklearn.metrics import accuracy_score

accuracy_score(model.predict(validation_set['text']), validation_set['is_mine'])  # accuracy on validation set. Very good!

0.9210526315789473

In [107]:
# this model can now be used to monitor timelines to spot if an author's style has changed or is being hacked